In [ ]:
import pandas as pd
import numpy as np

import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input

from dotenv import load_dotenv
import os

load_dotenv()

import functions as f

# Access the environment variables from the .env file
password = os.environ.get('DB_PASS')


# Note that when you use _SQLAlchemy_ and establish the connection, you do not even need to be logged in Sequel Pro or MySQL Workbench.

## Connect to DB

In [ ]:
# password = getpass.getpass()

bd = "building_permits"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)
engine

## Load Dataframe

In [ ]:
df = pd.read_csv("../data/clean/building_permits_addition_alteration_clean.csv")
display(df.head(1))

## Seed Data

### building_construction_types

In [ ]:
building_construction_types_unique = df["building_construction_type"].unique()
print(building_construction_types_unique)

def assign_index(row):
    return list(building_construction_types_unique).index(row["building_construction_type"])

df["building_construction_type_id"] = df.apply(assign_index, axis=1)

In [ ]:
building_construction_types_df = pd.DataFrame(df[["building_construction_type_id", "building_construction_type"]])
building_construction_types_df.drop_duplicates(inplace=True)
building_construction_types_df.rename(columns={'building_construction_type_id': 'id', 'building_construction_type': 'type'}, inplace=True)
building_construction_types_df

In [ ]:
with engine.connect() as connection:
    building_construction_types_df.to_sql("building_construction_type", connection, index=False, if_exists="replace")

In [ ]:
df["building_construction_type"] = df["building_construction_type_id"]
df.drop("building_construction_type_id", axis=1, inplace=True)

### Function to streamline preparing df for seeding

In [ ]:
# def prepare_for_seeding(df, column, db_column):
#     uniques = df[column].unique()
#     print(f"Unique values for {column}:", uniques)

#     def assign_index(row):
#         return list(uniques).index(row[column])

#     df[f"{column}_id"] = df.apply(assign_index, axis=1)
#     display(f"{column}_id", df[[f"{column}_id", column]])
#     prepared_df = pd.DataFrame(df[[f"{column}_id", f"{column}"]])
#     prepared_df.drop_duplicates(inplace=True)
#     prepared_df.rename(columns={f"{column}_id": 'id', column: db_column}, inplace=True)
        
#     return df, prepared_df

### building_use

In [ ]:
df, building_use_df = f.prepare_for_seeding(df, "building_use", "use")

In [ ]:
with engine.connect() as connection:
    building_use_df.to_sql("building_use", connection, index=False, if_exists="replace")

#### also for current_property_use

In [ ]:
building_use_unique = df["building_use"].unique()
print(building_use_unique)

def assign_index(row):
    return list(building_use_unique).index(row["current_property_use"])

df["current_property_use_id"] = df.apply(assign_index, axis=1)


In [ ]:
df["building_use"] = df["building_use_id"]
df.drop("building_use_id", axis=1, inplace=True)

In [ ]:
df["current_property_use"] = df["current_property_use_id"]
df.drop("current_property_use_id", axis=1, inplace=True)

### season

In [ ]:
df, season_df = f.prepare_for_seeding(df, "season", "name")

In [ ]:
with engine.connect() as connection:
    season_df.to_sql("season", connection, index=False, if_exists="replace")

In [ ]:
df["season"] = df["season_id"]
df.drop("season_id", axis=1, inplace=True)

### firm

In [ ]:
df, firm_df = f.prepare_for_seeding(df, "standardized_firm_name", "standardized_firm_name")

with engine.connect() as connection:
    firm_df.to_sql("firm", connection, index=False, if_exists="replace")

In [ ]:
df.columns

In [ ]:
df["standardized_firm_name"] = df["standardized_firm_name_id"]
df.drop(columns=["firm_name", "standardized_firm_name_id", "original_firm_name"], inplace=True)

### total_cost_bins

In [ ]:
df, total_cost_bins_df = f.prepare_for_seeding(df, "total_cost_bins", "bin_name")

with engine.connect() as connection:
    total_cost_bins_df.to_sql("total_cost_bins", connection, index=False, if_exists="replace")

In [ ]:
df["total_cost_bins"] = df["total_cost_bins_id"]
df.drop("total_cost_bins_id", axis=1, inplace=True)

### costs

In [ ]:
df_costs = df[[
    "record_number",
    "building_cost", 
    "electrical_cost", 
    "plumbing_cost",
    "gas_cost",
    "hvac_cost",
    "fire_prevention_cost", 
    "calc_total_cost"
]]

with engine.connect() as connection:
    df_costs.to_sql("costs", connection, index=False, if_exists="replace")

df_costs

In [ ]:
df.drop(columns=[
    "building_cost", 
    "electrical_cost", 
    "plumbing_cost",
    "gas_cost",
    "hvac_cost",
    "fire_prevention_cost", 
    "calc_total_cost"
    ], inplace=True)

df

### property

In [ ]:
# df_property = df[["record_number", "address", "latitude", "longitude"]]

# df_property.duplicated(subset="address").value_counts()

In [ ]:
property_unique = df["address"].unique()
print("Unique properties:", df["address"].nunique())

def assign_index(row):
    return list(property_unique).index(row["address"])

df["property"] = df.apply(assign_index, axis=1)

property_df = pd.DataFrame(df[["property", "address", "latitude", "longitude"]])
property_df.drop_duplicates(inplace=True)
property_df.rename(columns={'property': 'id'}, inplace=True)
property_df

In [ ]:
with engine.connect() as connection:
    property_df.to_sql("property", connection, index=False, if_exists="replace")

In [ ]:
df_property = df[[
    "record_number",
    "address", 
    "latitude", 
    "longitude"
]]

df.drop(columns=[
    "address", 
    "latitude", 
    "longitude"
    ], inplace=True)

df

### status

In [ ]:
df, df_status = f.prepare_for_seeding(df, "status", "value")

with engine.connect() as connection:
    df_status.to_sql("status", connection, index=False, if_exists="replace")

In [ ]:
df["status"] = df["status_id"]
df.drop("status_id", axis=1, inplace=True)

### keywords

In [ ]:
df_keyword_data = pd.read_csv("../data/clean/keyword_data.csv")
df_keyword_data

In [ ]:
df_unique_keywords = pd.read_csv("../data/clean/unique_keywords.csv")
df_unique_keywords

In [ ]:
df_unique_keywords.reset_index(inplace=True)
df_unique_keywords.rename(columns={"index": "keyword_id"}, inplace=True)

In [ ]:

df_keyword_merged = df_keyword_data.merge(df_unique_keywords[["keyword_id", "keyword"]], on="keyword")
df_keyword_merged

In [ ]:
df_keyword_sql = df_keyword_merged[["record_number", "keyword_id", "frequency"]]

with engine.connect() as connection:
    df_keyword_sql.to_sql("record_keyword", connection, if_exists="replace")

In [ ]:
df_unique_keywords.rename(columns={'keyword_id': 'id'}, inplace=True)

with engine.connect() as connection:
    df_unique_keywords.to_sql("keyword", connection, index=False, if_exists="replace")

### Main Dataframe (record)

In [ ]:
# df_merged = df.merge(df_keyword_sql[["record_number", "keyword_id"]], on="record_number")

In [ ]:
df.columns

In [ ]:
df.drop(columns=[
    'keywords',
    ], inplace=True)

In [ ]:
with engine.connect() as connection:
    df.to_sql("record", connection, index=False, if_exists="replace")